# Word Error Rate
## GENFI (ES - EN)
Vamos a sacar este parametro en base a codigo que vamos constuyendo y sacando de internet.

Primero vamos a sacar a los achivos de sus carpetas de la task force para ordenar esto.

In [2]:
import os
import shutil

def copiar_txt_desde_carpetas(lista_carpetas, carpeta_destino="/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/all_txt"):
    # Crear la carpeta de destino si no existe
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)

    for carpeta in lista_carpetas:
        if os.path.exists(carpeta):  # Verificamos que exista la carpeta
            for archivo in os.listdir(carpeta):
                if archivo.endswith(".txt"):
                    ruta_origen = os.path.join(carpeta, archivo)
                    ruta_destino = os.path.join(carpeta_destino, archivo)

                    # Evitamos sobreescribir si hay archivos con el mismo nombre
                    contador = 1
                    nombre_archivo, extension = os.path.splitext(archivo)
                    while os.path.exists(ruta_destino):
                        ruta_destino = os.path.join(
                            carpeta_destino, f"{nombre_archivo}_{contador}{extension}")
                        contador += 1

                    shutil.copy2(ruta_origen, ruta_destino)
                    print(f"✅ Copiado: {ruta_origen} -> {ruta_destino}")
        else:
            print(f"⚠️ La carpeta {carpeta} no existe.")

# 👉 USO:
# Lista de carpetas que querés procesar
carpetas_a_buscar = [
    "/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/Agus",
    "/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/Miguel",
    "/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/Rafa",
    "/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/Vanesa"
]

copiar_txt_desde_carpetas(carpetas_a_buscar)


✅ Copiado: /home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/Agus/01-167__FREE__transcription.txt -> /home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/all_txt/01-167__FREE__transcription.txt
✅ Copiado: /home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/Agus/07-008__FREE__transcription.txt -> /home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/all_txt/07-008__FREE__transcription.txt
✅ Copiado: /home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/Agus/01-126__FREE__transcription.txt -> /home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/all_txt/01-126__FREE__transcription.txt
✅ Copiado: /home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/Agus/07-011__FREE__transcription.txt -> /home/matias/Documents/UdeSA/CNC/diariziation_

Las carpetas quedan en:

- GROUND TRUTH: diariziation_error_rate/GENFI_Error_Rates/WER-ground-truth/all_txt
- PREDICTION: diariziation_error_rate/GENFI_Error_Rates/WER-prediction/all_txt

In [3]:
import os
import pandas as pd

# Rutas de las carpetas (modificá según corresponda)
carpeta_pred = "/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/all_txt"
carpeta_gt = "/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-ground-truth/all_txt"

# Inicializamos una lista donde guardaremos los registros
datos = []

# Recorremos todos los archivos de la carpeta de prediction
for archivo in os.listdir(carpeta_pred):
    if archivo.endswith(".txt"):
        id_archivo = os.path.splitext(archivo)[0]

        # Armamos las rutas de ambos archivos
        path_pred = os.path.join(carpeta_pred, archivo)
        path_gt = os.path.join(carpeta_gt, archivo)

        # Leemos prediction
        with open(path_pred, "r", encoding="utf-8") as f:
            prediction = f.read().strip()

        # Leemos ground_truth (si existe)
        if os.path.exists(path_gt):
            with open(path_gt, "r", encoding="utf-8") as f:
                ground_truth = f.read().strip()
        else:
            ground_truth = ""  # Si no existe, lo dejamos vacío

        # Guardamos los datos
        datos.append({
            "ID": id_archivo,
            "ground_truth": ground_truth,
            "prediction": prediction
        })

# Convertimos a DataFrame y exportamos a CSV
df = pd.DataFrame(datos)
df.to_csv("/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-comparacion.csv", index=False, encoding="utf-8")

print("✅ Archivo WER-comparacion.csv generado correctamente.")


✅ Archivo WER-comparacion.csv generado correctamente.


# WER Genfi

In [7]:
import os
from jiwer import wer
import pandas as pd


genfi_df = pd.read_csv("/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-comparacion.csv")


# calculate over all the dataset
wer_list = []
wer_dict = {}
for index, row in genfi_df.iterrows():
    try:
        reference = row["ground_truth"]
        hypothesis = row["prediction"]
        reference = reference.lower()
        hypothesis = hypothesis.lower()
        reference = reference.replace(".", "").replace(",", "").replace(";", "").replace(":", "").replace("!", "").replace("?", "").replace("¿", "").replace("¡", "")
        hypothesis = hypothesis.replace(".", "").replace(",", "").replace(";", "").replace(":", "").replace("!", "").replace("?", "")
        error = wer(reference, hypothesis)
        wer_list.append(error)
        wer_dict[row["ID"]] = error
    except AttributeError:
        print("ERROR: ", reference, hypothesis)
df = pd.DataFrame.from_dict(wer_dict, orient='index', columns=['WER'])
df.to_csv("/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WERs.csv", index=True, encoding="utf-8")
print("WER: ", sum(wer_list)/len(wer_list))


WER:  0.11722420765738159
